In [14]:
import os
import pandas as pd

#### GET BRENT EUR by CSV
df_brent_eur = pd.read_csv(
    "csv/brent_eur_liter_monthly.csv",
    parse_dates=["Date_monthly"]
)
df_brent_eur.head(2)

,Date_monthly,brent_gazole_EUR_liter
0,2007-07-01,0.3839
1,2007-08-01,0.3817


In [15]:
#### GET REFINING MERGING BRENT EUR by CSV
df_refining_eur = pd.read_csv(
    "csv/refining_margin_brent_eur_liter_monthly.csv",
    parse_dates=["Date_monthly"]
)
df_refining_eur.head(2)

,Date_monthly,refining_margin_brent_eur_liter
0,2015-01-01,0.03633
1,2015-02-01,0.03900


In [16]:
#### GET TRANSPORTATION FEES GAZOLE EUR by CSV
df_transport_fees_eur = pd.read_csv(
    "csv/transport_fees_eur_liter.csv",
    parse_dates=["Date_monthly"]
)
df_transport_fees_eur.head(2)

,Date_monthly,Gazole_transportation_fees_eur_liter
0,2015-01-01,0.086
1,2015-02-01,0.086


In [17]:
#### GET HTT TTC GAZOLE EUR by CSV
df_gazole_eur = pd.read_csv(
    "csv/gazole_htt_ttc_eur_liter_monthly.csv",
    parse_dates=["Date_monthly"]
)
df_gazole_eur.head(2)

,Date_monthly,gazole_eur_liter_htt,gazole_eur_liter_ttc
0,2015-01-01,0.4484,1.1149
1,2015-02-01,0.5081,1.1865


In [18]:
# CREATE GLOBAL TABLE
df_merged = pd.merge(df_gazole_eur, df_brent_eur, on='Date_monthly', how='left')
df_merged = pd.merge(df_merged, df_refining_eur, on='Date_monthly', how='left')
df_merged = pd.merge(df_merged, df_transport_fees_eur, on='Date_monthly', how='left')
df_merged.head(5)

,Date_monthly,gazole_eur_liter_htt,gazole_eur_liter_ttc,brent_gazole_EUR_liter,refining_margin_brent_eur_liter,Gazole_transportation_fees_eur_liter
0,2015-01-01,0.4484,1.1149,0.3223,0.03633,0.086
1,2015-02-01,0.5081,1.1865,0.3355,0.03900,0.086
2,2015-03-01,0.5251,1.2069,0.3696,0.04791,0.086
3,2015-04-01,0.5205,1.2014,0.3674,0.04180,0.086
4,2015-05-01,0.5477,1.2341,0.4103,0.03840,0.086


In [23]:
# VERIFY OFFICIAL HTT IS ACCURATE TO CALCULATE HTT
# HT = Brent + refining_margin + transportation_fees
df_merged["calculate_gazole_eur_liter_htt"] = (
df_merged["brent_gazole_EUR_liter"] + df_merged["refining_margin_brent_eur_liter"] + df_merged["Gazole_transportation_fees_eur_liter"]
)

# Create special col for gazole taxes
df_merged["gazole_taxes_eur_liter"] = df_merged["gazole_eur_liter_ttc"] - df_merged["gazole_eur_liter_htt"]
df_merged.head(5)

,Date_monthly,gazole_eur_liter_htt,gazole_eur_liter_ttc,brent_gazole_EUR_liter,refining_margin_brent_eur_liter,Gazole_transportation_fees_eur_liter,calculate_gazole_eur_liter_htt,gazole_taxes,gazole_taxes_eur_liter
0,2015-01-01,0.4484,1.1149,0.3223,0.03633,0.086,0.44463,0.6665,0.6665
1,2015-02-01,0.5081,1.1865,0.3355,0.03900,0.086,0.46050,0.6784,0.6784
2,2015-03-01,0.5251,1.2069,0.3696,0.04791,0.086,0.50351,0.6818,0.6818
3,2015-04-01,0.5205,1.2014,0.3674,0.04180,0.086,0.49520,0.6809,0.6809
4,2015-05-01,0.5477,1.2341,0.4103,0.03840,0.086,0.53470,0.6864,0.6864


In [24]:
# Export CSV
os.makedirs("csv", exist_ok=True)
df_merged.to_csv("csv/df_table_all_datas.csv", index=False)